# Generate Bollinger bands using dedicated service - Python

### Overview
Bollinger bands were created by John Bollinger, they are envelopes and plotted around the moving average. The width of the bands is based on the standard deviaton of the closing prices from a moving average. 

The purpose of Bollinger Bands is to provide a relative definition of high and low prices of a market. By definition, prices are high at the upper band and low at the lower band.

This sample demonstrates how to request and plot from a dedicated data service **on-demand** Bollinger bands.

### Inputs/outputs
Bollinger bands indicator requires instrument's identifier,date time intervals and number of periods as per inputs and returns data points for the following components:
1. *Middle band* = n-period moving average
2. *Upper band* = Middle band + (k * n-period standard deviation)
3. *Lower band* = Middle band - (k * n-period standard deviation)

Where:
* *n: number of periods*
* *k: factor to apply to the standard deviation value, k=2 as default value*

This sample shows how to request and plot a simple Bollinger bands graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve Bollinger bands from the hosted service. The queried endpoint in this script are:
* *TickBollingerService*: to directly retrieve bollinger bands tick data reponse from the server

### Modules required
1. Systemathics packages:
    * *systemathics.apis.services.tick_analytics.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *matpotlib* as per display package

***

# Run Bollinger bands sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib plotly systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
from datetime import timedelta
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick_analytics.v1.tick_bollinger_pb2 as tick_bollinger
import systemathics.apis.services.tick_analytics.v1.tick_bollinger_pb2_grpc as tick_bollinger_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request
To request *tick Bollinger bands* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick Bollinger bands request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'BATS'

#### 3.2 Bollinger bands parameters

In [ ]:
field = tick_bollinger.BOLLINGER_PRICE_TRADE
length = 20
deviation = 0.4
sampling = 1 * 60

#### 3.2 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
today = datetime.today()
start = today - timedelta(days=50)

date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = start.year, month = start.month, day = start.day), 
    end_date = date.Date(year = today.year, month = today.month, day = today.day)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 14, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 17, minutes = 0, seconds = 0)
)

#### 3.3 Request creation
The following code snippets creates *gRPC client*, process request and returns the request reply:

In [ ]:
# generate constraints based on the previous time selection
constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

In [ ]:
# generate the tick Bollinger request
request = tick_bollinger.TickBollingerRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker), 
    constraints = constraints,
    field = field,
    length = length,
    deviation = deviation,
    sampling = duration.Duration(seconds = sampling),
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick Bollinger service
        service = tick_bollinger_service.TickBollingerServiceStub(channel)
        
        # process the tick Bollinger request
        time_stamps, uppers, middles, lowers, prices = [],[],[],[],[]
        metadata = [('authorization', token)]
        for bb in service.TickBollinger(request=request, metadata=metadata):
            time_stamps.append(bb.time_stamp)
            prices.append(bb.value)    
            if bb.middle.value > 0:  # we have Bollinger values
                middles.append(bb.middle.value)
                uppers.append(bb.upper.value)
                lowers.append(bb.lower.value)
            else: 
                middles.append(None)
                uppers.append(None)
                lowers.append(None)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

The following code snippet displays the total retrieved Bollinger bands data points count:

In [ ]:
print('Total Bollinger bands data points retrieved: {0}'.format(len(time_stamps)))

### Step 4: Visualize data

#### 4.1 Retrieve Bollinger bands data
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
# create the dataframe for tick Bollinger bands data
dates=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps]
d = {'Date': dates, 'Upper': uppers, 'Middle': middles, 'Lower' : lowers, 'Price': prices}
df = pd.DataFrame(data=d)
df

#### 4.2 Plot Bollinger bands with matplotlib

In [ ]:
# select number of ticks
N = 150

In [ ]:
import matplotlib.pyplot as plt

# create figure and axis objects with subplots()
fig,ax = plt.subplots(1,1,figsize=(25,10))
ax.plot('Date', 'Upper', data=df.tail(N), marker='', color='purple', linewidth=2, label="Upper")
ax.plot('Date', 'Middle', data=df.tail(N), marker='', color='orange', linewidth=2, label="Middle")
ax.plot('Date', 'Lower', data=df.tail(N), marker='', color='green', linewidth=2, label="Lower")
ax.plot('Date', 'Price', data=df.tail(N), marker='', color='blue', linewidth=2, label="Price")

ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Price",color="black",fontsize=14)
plt.legend()
plt.title('Tick Bollinger bands and prices for {0}-{1}: deviation = {2}, length = {3} | Last {4} ticks'.format(ticker,exchange,deviation,length,N))
plt.show()

#### 4.3 Plot Bollinger bands with plotly

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Upper'], name='Upper', marker_color ='green'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Middle'], name='Middle', marker_color ='orange'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Lower'], name='Lower', marker_color ='red'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], name='Price', marker_color ='blue'))
fig.update_xaxes(title_text = 'Date')
fig.update_layout(title = 'Tick Bollinger bands and prices for {0}-{1}: deviation = {2} | length = {3}'.format(ticker,exchange,deviation,length))
fig.show()